# Create a tasking order

- <a href="#step-1">Step 1. Set up the notebook</a>
- <a href="#step-2">Step 2. Get access</a>
- <a href="#step-3">Step 3. Create an order</a>
- <a href="#step-4">Step 4. Review feasibility study options</a>
- <a href="#step-5">Step 5. Activate your order</a>
- <a href="#step-6">Step 6. Monitor orders</a>
- <a href="#step-7">Step 7. Download assets</a>

## <a id="step-1"></a> Step 1. Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, geojson, pathlib
import pandas as pd

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

1. Click the link above to the created file and paste the following code:
    ```
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

## <a id="step-2"></a> Step 2. Get access

### 1. Select a tasking collection

Choose a tasking collection and get its `data_product_id` for ordering.

In [ ]:
tasking_collections = up42.CollectionType.TASKING
sort_by_name = up42.CollectionSorting.name
data_collections = up42.ProductGlossary.get_collections(
    collection_type=tasking_collections,
    sort_by=sort_by_name
)
pd.DataFrame({"collection": c.name, "product": d.name} for c in data_collections for d in c.data_products)

Choose a data product and copy its ID.

In [ ]:
# The following value is the product id for Beijing-3N — Custom:
data_product_id = "306153d7-7c44-4621-9200-379c7f6cd649"

### 2. Request access

If you want to order the chosen collection for the first time, you need to [request access to it](https://docs.up42.com/getting-started/restrictions#request-access).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

### 3. Accept the EULA

If you want to order the chosen collection for the first time, you need to accept its [end-user license agreement (EULA)](https://docs.up42.com/getting-started/account/eulas).

## <a id="step-3"></a> Step 3. Create an order

### 1. Fill out an order form

Get detailed information about the parameters needed to create an order for the chosen data product.

In [ ]:
data_product_id.schema

Before creating an order, find out how order parameters affect the [timeframe of your tasking order](https://docs.up42.com/data/tasking/acquisition).

Specify the geometry and use the required request body schema format for the chosen data product.

In [7]:
aoi_geometry = {
    "type": "Polygon",
    "coordinates": (
        (
            (13.375966, 52.515068),
            (13.375966, 52.516639),
            (13.378314, 52.516639),
            (13.378314, 52.515068),
            (13.375966, 52.515068),
        ),
    ),
}

features = geojson.FeatureCollection(features=[geojson.Feature(geometry=aoi_geometry)])

order_template = up42.BatchOrderTemplate(
    data_product_id=data_product_id,
    display_name=f"My tasking order-{date.today()}",
    features=features,
    params={
        "acquisitionStart": "2025-01-01",
        "acquisitionEnd": "2025-02-20",
        "acquisitionMode": "mono",
        "maxCloudCover": 10,
        "maxIncidenceAngle": 10,
        "geometricProcessing": "georectified",
        "pixelCoding": "16bit",
        "spectralBands": "pansharpened_3_band_true_color",
        "priority": "standard",
        "radiometricProcessing": "toa",
    },
    tags=["project-7", "optical"]
)

Check the parameters by printing them.

In [ ]:
order_template

### 2. Get a cost estimate

Get a cost estimation before creating an order.

In [ ]:
order_template.estimate

The response returns the overall credit amount that will be deducted from your credit balance if you decide to proceed with the ordering.

### 3. Create an order

Create an order with the requested parameters.

In [10]:
order_references = order_template.place()
order = order_references[0].order
order

If you've defined an AOI with multiple geometries, each geometry will create a separate order. Order names will be suffixed with incrementing numbers: `Order 1`, `Order 2`, etc.

Once the order is created, you will get a response like this:
```
2023-07-13 13:22:10,677 - Order is PLACED
2023-07-13 13:22:10,677 - Order your-order-id is now PLACED.
```

## <a id="step-4"></a> Step 4. Review feasibility study options

### 1. Review options

After an order is created, the Operations team will conduct a [feasibility study](https://docs.up42.com/data/tasking/timeframes/feasibility). They will evaluate the tasking parameters with the provider, and then will present the following assessments:

- The order is possible with the given parameters.
- The order requires modifications with suitable options proposed.

You will receive an email notifying you when your order feasibility has been assessed. Using your order ID, review the proposed feasibility options.

In [11]:
tasking.get_feasibility(order_id="68567134-27ad-7bd7-4b65-d61adb11fc78")

If none of the options are suitable, [contact](https://up42.com/company/contact-ordering) the Operations team.

### 2. Accept an option

You can't modify your order after accepting an option.

In [12]:
tasking.choose_feasibility(
    feasibility_id="68567134-27ad-7bd7-4b65-d61adb11fc78",
    accepted_option_id="a0d443a2-41e8-4995-8b54-a5cc4c448227"
)

## <a id="step-5"></a> Step 5. Activate an order

After accepting a feasibility study option, you will receive a price quote for your order. Using your order ID, review the received quotation.

In [13]:
quotation = up42.Quotation(order_id="68567134-27ad-7bd7-4b65-d61adb11fc78", )

To activate your order, accept the quotation. Once you accept the tasking quotation, the credits for that order will be deducted. The transaction can’t be reversed.

In [14]:
quotation.accept()
quotation.save()

## <a id="step-6"></a> Step 6. Monitor orders

Check the [status](https://docs.up42.com/developers/api-tasking#order-statuses) of your order.

In [15]:
order.status

You can also track the order status until the order is completed.

In [16]:
order.track()

## <a id="step-7"></a> Step 7. Download assets

The following statuses mean you can download assets from storage:
- `BEING_FULFILLED`: Some order assets might have been delivered. [Download](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-storage-assets.ipynb) these assets using the `Storage` class.
- `FULFILLED`: All order assets have been delivered. [Download](https://github.com/up42/documentation-notebooks/blob/main/storage/sdk-order-assets.ipynb) these assets using the `Order` class.